# Introduction
Build a multi-step, intelligent query-handling agent using LangGraph and Gemini 1.5 Flash.
- incoming query is passed through a series of purposeful nodes:
  - routing
  - analysis
  - research
  - response generation
  - validation

- we create a looping system that can re analyze and improve its output until the response is validated as complete or a max iteration threshold is reached.

# Installing required packages

In [1]:
! pip install langgraph langchain-google-genai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requ

In [ ]:
import os
from typing import Dict, Any, List
from dataclasses import dataclass
from langgraph.graph import Graph, StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage, SystemMessage
import json


os.environ["GOOGLE_API_KEY"] = "AIz*****************************IUA"

In [3]:
# declare the state
@dataclass
class AgentState:
    """State shared across all nodes in the graph"""
    query: str = ""
    context: str = ""
    analysis: str = ""
    response: str = ""
    next_action: str = ""
    iteration: int = 0
    max_iterations: int = 3

Nodes:
- router_node : Route and categorize the incoming query
- analyzer_node: Analyze the query and determine the approach
- researcher_node: Conduct additional research or information gathering
- responder_node: Generate the final response
- validator_node:

In [13]:
# Agent
class GraphAIAgent:
    def __init__(self):

        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0.7,
            convert_system_message_to_human=True
        )

        self.analyzer = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0.3,
            convert_system_message_to_human=True
        )

        self.graph = self._build_graph()

    def _build_graph(self) -> StateGraph:
        """Build the LangGraph workflow"""
        workflow = StateGraph(AgentState)

        workflow.add_node("router", self._router_node)
        workflow.add_node("analyzer", self._analyzer_node)
        workflow.add_node("researcher", self._researcher_node)
        workflow.add_node("responder", self._responder_node)
        workflow.add_node("validator", self._validator_node)

        workflow.set_entry_point("router")
        workflow.add_edge("router", "analyzer")
        workflow.add_conditional_edges(
            "analyzer",
            self._decide_next_step,
            {
                "research": "researcher",
                "respond": "responder"
            }
        )
        workflow.add_edge("researcher", "responder")
        workflow.add_edge("responder", "validator")
        workflow.add_conditional_edges(
            "validator",
            self._should_continue,
            {
                "continue": "analyzer",
                "end": END
            }
        )

        return workflow.compile()

    def _router_node(self, state: AgentState) -> Dict[str, Any]:
        """Route and categorize the incoming query"""
        system_msg = """You are a query router. Analyze the user's query and provide context that the patients are looking for from the disease.
        Determine and note all the key important parameters that the patient need to know."""

        messages = [
            SystemMessage(content=system_msg),
            HumanMessage(content=f"Query: {state.query}")
        ]

        response = self.llm.invoke(messages)
        return {
            "context": response.content,
            "iteration": state.iteration + 1
        }

    def _analyzer_node(self, state: AgentState) -> Dict[str, Any]:
        """Analyze the query and determine the approach"""
        system_msg = """Analyze the query and context. Determine if additional research is needed
        or if you can provide a direct response. Be thorough in your analysis."""

        messages = [
            SystemMessage(content=system_msg),
            HumanMessage(content=f"""
            Query: {state.query}
            Context: {state.context}
            Previous Analysis: {state.analysis}
            """)
        ]

        response = self.analyzer.invoke(messages)
        analysis = response.content

        if "research" in analysis.lower() or "more information" in analysis.lower():
            next_action = "research"
        else:
            next_action = "respond"
        return {
            "analysis": analysis,
            "next_action": next_action
        }

    def _researcher_node(self, state: AgentState) -> Dict[str, Any]:
        """Conduct additional research or information gathering"""
        system_msg = """You are a research assistant. Based on the analysis, gather relevant
        information and insights to help answer the query comprehensively."""

        messages = [
            SystemMessage(content=system_msg),
            HumanMessage(content=f"""
            Query: {state.query}
            Analysis: {state.analysis}
            Research focus: Provide detailed information relevant to the query.
            """)
        ]

        response = self.llm.invoke(messages)

        updated_context = f"{state.context}\n\nResearch: {response.content}"
        return {"context": updated_context}

    def _responder_node(self, state: AgentState) -> Dict[str, Any]:
        """Generate the final response"""
        system_msg = """You are a helpful AI assistant. Provide a comprehensive, accurate,
        and well-structured response based on the analysis and context provided."""

        messages = [
            SystemMessage(content=system_msg),
            HumanMessage(content=f"""
            Query: {state.query}
            Context: {state.context}
            Analysis: {state.analysis}

            Provide a complete and helpful response.
            """)
        ]

        response = self.llm.invoke(messages)
        return {"response": response.content}

    def _validator_node(self, state: AgentState) -> Dict[str, Any]:
        """Validate the response quality and completeness"""
        system_msg = """Evaluate if the response adequately answers the query.
        Return 'COMPLETE' if satisfactory, or 'NEEDS_IMPROVEMENT' if more work is needed."""

        messages = [
            SystemMessage(content=system_msg),
            HumanMessage(content=f"""
            Original Query: {state.query}
            Response: {state.response}

            Is this response complete and satisfactory?
            """)
        ]

        response = self.analyzer.invoke(messages)
        validation = response.content

        return {"context": f"{state.context}\n\nValidation: {validation}"}

    def _decide_next_step(self, state: AgentState) -> str:
        """Decide whether to research or respond directly"""
        return state.next_action

    def _should_continue(self, state: AgentState) -> str:
        """Decide whether to continue iterating or end"""
        if state.iteration >= state.max_iterations:
            return "end"
        if "COMPLETE" in state.context:
            return "end"
        if "NEEDS_IMPROVEMENT" in state.context:
            return "continue"
        return "end"

    def run(self, query: str) -> str:
        """Run the agent with a query"""
        initial_state = AgentState(query=query)
        result = self.graph.invoke(initial_state)
        return result["response"]

In [14]:
def main():
    agent = GraphAIAgent()

    test_queries = [
        "Explain quantum computing and its applications",
        "What are the best practices for machine learning model deployment?",
        "Create a story about a robot learning to paint"
    ]

    print("🤖 Graph AI Agent with LangGraph and Gemini")
    print("=" * 200)

    for i, query in enumerate(test_queries, 1):
        print(f"\n📝 Query {i}: {query}")
        print("-" * 30)

        try:
            response = agent.run(query)
            print(f"🎯 Response: {response}")
        except Exception as e:
            print(f"❌ Error: {str(e)}")

        print("\n" + "="*50)


if __name__ == "__main__":
    main()

🤖 Graph AI Agent with LangGraph and Gemini

📝 Query 1: Explain quantum computing and its applications
------------------------------


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_hu

🎯 Response: Quantum computing is a revolutionary field that leverages the principles of quantum mechanics to solve problems intractable for even the most powerful classical computers.  Unlike classical computers that store information as bits representing either 0 or 1, quantum computers use **qubits**.  The key difference lies in the unique properties of qubits:

* **Superposition:** A qubit can exist in a superposition, simultaneously representing 0, 1, or a combination of both.  This allows quantum computers to explore many possibilities concurrently, unlike classical computers which process one possibility at a time.

* **Entanglement:**  Multiple qubits can be entangled, meaning their fates are intertwined.  Measuring the state of one instantly reveals the state of the others, regardless of the distance separating them. This enables powerful parallel processing capabilities.

These properties enable quantum computers to tackle complex problems exponentially faster than classical c

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_hu

🎯 Response: The best practices for machine learning model deployment are highly context-dependent, varying significantly based on factors like model type, target environment, scale, data characteristics, and performance requirements.  There's no single "best" approach; the optimal strategy depends on a careful consideration of these factors.  However, several overarching principles apply across most scenarios.  We'll address these principles, followed by specific considerations for high-stakes applications like those in healthcare.

**I. Pre-Deployment Best Practices:**

* **Model Selection and Evaluation:**  Begin by selecting the appropriate model type (e.g., regression, classification, clustering) based on the problem.  Rigorously evaluate its performance using relevant metrics (accuracy, precision, recall, F1-score, AUC, etc.) on a held-out test set. Techniques like k-fold cross-validation help ensure robustness.  Document all evaluation steps thoroughly.

* **Model Versioning and 

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:424: UserWarning: Convert_system_message_to_hu

🎯 Response: Here's a story about a robot learning to paint:

The whirring of Unit 734, nicknamed "Brushbot" by its creator, Dr. Aris Thorne, filled the small, cluttered studio.  Brushbot, a marvel of engineering, possessed incredibly precise manipulators, perfect for assembling microchips.  But Dr. Thorne, a whimsical soul, saw something more in his creation – a potential artist.

He loaded Brushbot's core programming with vast datasets of art history, from cave paintings to modern masterpieces. He taught it color theory, composition, and the techniques of various artistic movements.  Brushbot diligently absorbed this information, its optical sensors analyzing every brushstroke, every hue, every shade.  Its first attempts were… predictable.  Geometric landscapes rendered with flawless precision, but devoid of emotion or soul.  Portraits that captured every wrinkle and freckle, yet missed the sparkle in the subject's eyes.

Dr. Thorne remained patient.  He knew that true art wasn't just